In [1]:
import mmap
import os

In [2]:
file = "../data/smallTwitter.json"
size = os.stat(file).st_size 
file, size

('../data/smallTwitter.json', 242534080)

In [5]:
%%time

lines = []
with open(file, 'rb') as f:
    f.seek(0)
    while f.tell() < size:
        lines.append(f.readline())

CPU times: user 2.92 s, sys: 1.77 s, total: 4.69 s
Wall time: 9.27 s


In [6]:
len(lines)

9229442

In [56]:
242534080//2

121267040

In [26]:
%%time
lines0 = []

with open(file, 'rb') as f:
    f.seek(size // 2)
    mmapped_data = mmap.mmap(f.fileno(), length=size // 2, access=mmap.ACCESS_READ)

    line = mmapped_data.readline()
    while line:
        lines0.append(line)
        line = mmapped_data.readline()
    
    mmapped_data.close()

CPU times: user 680 ms, sys: 433 ms, total: 1.11 s
Wall time: 3.42 s


In [28]:
len(lines0)

4583006

In [44]:
size / 1024 / 1024

231.29852294921875

In [46]:
512 * 1024 * 1024

536870912

In [43]:
512 * 1024 * 1024 > size

True

In [47]:
lines = [] 

def read_512mb_into_memory(file, lines, chunk_start, chunk_end):
    fsize = os.stat(file).st_size
    MMAP_SIZE = 536870912
    length = MMAP_SIZE if fsize < MMAP_SIZE else fsize
    
    with open(file, mode='rb') as f:
        mmaped_data = mmap.mmap(f.fileno(), length=MMAP_SIZE, offset=536870912,access=mmap.ACCESS_READ)

        line = mmaped_data.readline()
        while line:
            lines.append(line)
            line = mmaped_data.readline()

        mmaped_data.close()
    
read_512mb_into_memory(file, lines, )

TypeError: read_512mb_into_memory() missing 1 required positional argument: 'chunk_end'

In [42]:
lines[0:5], len(lines)

([b'            "start": 0,\n',
  b'            "end": 10,\n',
  b'            "username": "isobelroe",\n',
  b'            "id": "28537321"\n',
  b'          }\n'],
 20320215)

In [51]:
file_size = os.path.getsize(file)

# 计算 bin 的数量
num_bins = file_size // (512 * 1024 * 1024)

# 如果文件大小不能被 bin_size 整除，则最后一个 bin 的大小会小于 bin_size
if file_size % (512 * 1024 * 1024) != 0:
    num_bins += 1

In [52]:
num_bins

1

In [87]:
def split_file_into_n_chunks(filename, num_chunks):
    file_size = os.path.getsize(filename)
    chunk_size = file_size // num_chunks

    if file_size % num_chunks != 0:
        last_chunk_size = file_size - chunk_size * (num_chunks - 1)

    chunk_ranges = []

    for i in range(num_chunks):
        start = i * chunk_size
        if i == num_chunks - 1 and last_chunk_size:
            end = start + last_chunk_size
        else:
            end = start + chunk_size

        chunk_ranges.append((start, end))

    return chunk_ranges

In [88]:
split_file_into_n_chunks('../data/bigTwitter.json', 8)

[(0, 2341913382),
 (2341913382, 4683826764),
 (4683826764, 7025740146),
 (7025740146, 9367653528),
 (9367653528, 11709566910),
 (11709566910, 14051480292),
 (14051480292, 16393393674),
 (16393393674, 18735307060)]

In [61]:
def read_file_with_bins(filename, bin_size=512):
    file_size = os.path.getsize(filename)
    num_bins = file_size // (bin_size * 1024 * 1024)

    if file_size % (bin_size * 1024 * 1024) != 0:
        num_bins += 1

    bin_ranges = []
    for i in range(num_bins):
        start = i * bin_size * 1024 * 1024
        end = min(start + bin_size * 1024 * 1024, file_size)
        bin_ranges.append((start, end))

    return bin_ranges

In [90]:
%%time
lines = [] 

def read_into_memory(file, lines, chunk_start, chunk_end):
    fsize = os.stat(file).st_size
    length = chunk_end - chunk_start
    
    with open(file, mode='rb') as f:
        mmaped_data = mmap.mmap(f.fileno(), length=length, offset=chunk_start,access=mmap.ACCESS_READ)

        line = mmaped_data.readline()
        while line:
            lines.append(line)
            line = mmaped_data.readline()

        mmaped_data.close()
    
read_512mb_into_memory('../data/bigTwitter.json', lines, 536870912, 1073741824)

CPU times: user 2.47 s, sys: 1.59 s, total: 4.06 s
Wall time: 14.1 s


In [63]:
len(lines), lines[0:5]

(20320215,
 [b'            "start": 0,\n',
  b'            "end": 10,\n',
  b'            "username": "isobelroe",\n',
  b'            "id": "28537321"\n',
  b'          }\n'])

In [93]:
def read_into_memory(file, lines, chunk_start, chunk_end):
    length = chunk_end - chunk_start
    
    with open(file, mode='rb') as f:
        mmaped_data = mmap.mmap(f.fileno(), length=length, offset=chunk_start,access=mmap.ACCESS_READ)

        line = mmaped_data.readline()
        while line:
            lines.append(line)
            line = mmaped_data.readline()

        mmaped_data.close()

In [ ]:
%%timeit

bins = read_file_with_bins('../data/bigTwitter.json')
lines = list()
for chunk_start, chunk_end in bins:
    read_into_memory('../data/bigTwitter.json', lines, chunk_start, chunk_end)
    
len(lines), lines[0:5]

In [66]:
%%time
lines = []
with open('../data/bigTwitter.json', 'rb') as f:
    line = f.readline()
    
    while :
        line = f.readline()
        lines.append(line)

len(lines), lines[0:5]       

CPU times: user 7.32 s, sys: 3.76 s, total: 11.1 s
Wall time: 21.7 s


(20320215,
 [b'            "start": 0,\n',
  b'            "end": 10,\n',
  b'            "username": "isobelroe",\n',
  b'            "id": "28537321"\n',
  b'          }\n'])

In [92]:
len(read_file_with_bins('../data/bigTwitter.json'))

35